In [2]:
from heapq import heappush, heappop

# Goal state for reference
GOAL_STATE = (1, 2, 3,
              8, 0, 4,
              7, 6, 5)  # 0 is the blank tile

# Directions for moving the blank tile (row, col)
DIRECTIONS = [(-1, 0), (1, 0), (0, -1), (0, 1)]


def misplaced_tiles(state, goal=GOAL_STATE):
    """Heuristic: Number of misplaced tiles."""
    return sum(1 for i in range(9) if state[i] != 0 and state[i] != goal[i])


def manhattan_distance(state, goal=GOAL_STATE):
    """Heuristic: Sum of Manhattan distances of tiles from their goal positions."""
    distance = 0
    for tile in range(1, 9):  # skip blank tile
        current_index = state.index(tile)
        goal_index = goal.index(tile)
        current_row, current_col = divmod(current_index, 3)
        goal_row, goal_col = divmod(goal_index, 3)
        distance += abs(current_row - goal_row) + abs(current_col - goal_col)
    return distance


def get_neighbors(state):
    """Generate all valid neighbor states from the current state."""
    neighbors = []
    zero_index = state.index(0)
    zero_row, zero_col = divmod(zero_index, 3)

    for dr, dc in DIRECTIONS:
        new_row, new_col = zero_row + dr, zero_col + dc
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_index = new_row * 3 + new_col
            new_state = list(state)
            # Swap blank with adjacent tile
            new_state[zero_index], new_state[new_index] = new_state[new_index], new_state[zero_index]
            neighbors.append(tuple(new_state))
    return neighbors


def reconstruct_path(came_from, current):
    """Reconstruct the path from start to goal."""
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path


def a_star(start_state, heuristic_func):
    open_list = []
    heappush(open_list, (0 + heuristic_func(start_state), 0, start_state))  # (f_score, g_score, state)
    came_from = {}

    g_score = {start_state: 0}
    closed_set = set()

    while open_list:
        f, g, current = heappop(open_list)

        if current == GOAL_STATE:
            return reconstruct_path(came_from, current)

        closed_set.add(current)

        for neighbor in get_neighbors(current):
            if neighbor in closed_set:
                continue

            tentative_g_score = g + 1  # Each move costs 1

            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score = tentative_g_score + heuristic_func(neighbor)
                heappush(open_list, (f_score, tentative_g_score, neighbor))

    return None  # No solution found


# Example usage:
if __name__ == "__main__":
    start = (2, 8, 3,
             1, 6, 4,
             7, 0, 5)

    print("Solving with Misplaced Tiles Heuristic...")
    path1 = a_star(start, misplaced_tiles)
    print(f"Number of moves: {len(path1) - 1}")
    for state in path1:
        print(state[:3])
        print(state[3:6])
        print(state[6:])
        print("---")

    print("\nSolving with Manhattan Distance Heuristic...")
    path2 = a_star(start, manhattan_distance)
    print(f"Number of moves: {len(path2) - 1}")
    for state in path2:
        print(state[:3])
        print(state[3:6])
        print(state[6:])
        print("---")

Solving with Misplaced Tiles Heuristic...
Number of moves: 5
(2, 8, 3)
(1, 6, 4)
(7, 0, 5)
---
(2, 8, 3)
(1, 0, 4)
(7, 6, 5)
---
(2, 0, 3)
(1, 8, 4)
(7, 6, 5)
---
(0, 2, 3)
(1, 8, 4)
(7, 6, 5)
---
(1, 2, 3)
(0, 8, 4)
(7, 6, 5)
---
(1, 2, 3)
(8, 0, 4)
(7, 6, 5)
---

Solving with Manhattan Distance Heuristic...
Number of moves: 5
(2, 8, 3)
(1, 6, 4)
(7, 0, 5)
---
(2, 8, 3)
(1, 0, 4)
(7, 6, 5)
---
(2, 0, 3)
(1, 8, 4)
(7, 6, 5)
---
(0, 2, 3)
(1, 8, 4)
(7, 6, 5)
---
(1, 2, 3)
(0, 8, 4)
(7, 6, 5)
---
(1, 2, 3)
(8, 0, 4)
(7, 6, 5)
---
